<a href="https://www.kaggle.com/code/nancymee/credit-card-fraud-detection?scriptVersionId=189157293" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [42]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [43]:
data = pd.read_csv('/kaggle/input/credit-card-fraud-data/creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [44]:
data.shape

(284807, 31)

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

## Categorical and Numerical Features

In [46]:
num_feat = [feature for feature in data.columns if data[feature].dtype != 'O']
cat_feat = [feature for feature in data.columns if data[feature].dtype == 'O']

In [47]:
num_feat

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount',
 'Class']

In [48]:
cat_feat

[]

### Missing Values

In [49]:
total=data.isnull().sum().sort_values(ascending=False)
percent=(data.isnull().sum()/data.isnull().count()*100).sort_values(ascending=False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

,Total,Percent
Time,0,0.0
V16,0,0.0
Amount,0,0.0
V28,0,0.0
V27,0,0.0
V26,0,0.0
V25,0,0.0
V24,0,0.0
V23,0,0.0
V22,0,0.0


"Class" is the target variable. 
* Legit transactions = 0
* Fraud transactions = 1

In [50]:
data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

There are 492 fraud transactions.

## Data Cleaning and Preprocessing

In [51]:
legit = data[data.Class == 0]
fraud = data[data.Class == 1]

In [52]:
legit.shape,fraud.shape

((284315, 31), (492, 31))

In [53]:
data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [54]:
legit_sample = legit.sample(n=492)
legit_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
217018,140728.0,-0.111629,0.619835,-0.004133,-0.063503,1.504045,-0.317948,0.840894,-0.189993,-0.080063,...,-0.345524,-0.892542,-0.054848,-0.097943,-0.596432,0.315761,-0.054834,-0.033327,1.98,0
210523,138020.0,0.094823,0.943427,-0.380045,-0.952696,1.655329,0.144880,1.332652,-0.463352,0.107920,...,0.240513,1.368728,-0.292101,-0.146433,-0.667049,-0.269276,0.288554,-0.073472,0.99,0
9387,13746.0,1.109824,0.331352,1.560526,2.719556,-0.483555,0.712093,-0.741679,0.236268,1.119022,...,-0.204147,-0.206828,0.000416,-0.067350,0.297569,-0.079282,0.013765,0.016206,0.00,0
46887,42975.0,0.443771,-2.268674,1.139574,0.125735,-2.150027,0.404790,-0.845121,0.114456,0.287907,...,0.403281,0.579466,-0.367342,0.502044,0.204558,-0.156440,0.017935,0.101540,426.72,0
96062,65591.0,1.209410,-0.441473,-0.984072,-1.041218,1.687524,3.300782,-0.702919,0.835463,0.174765,...,-0.263361,-1.067383,0.089452,1.032108,0.138759,0.752848,-0.076037,0.017357,64.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186137,126971.0,1.957503,-0.365742,-0.422290,0.185716,-0.438269,-0.032136,-0.672475,0.206180,1.031679,...,-0.097386,-0.265410,0.419854,0.633988,-0.501447,-0.662194,0.022970,-0.036143,1.18,0
263041,160771.0,1.922122,-0.357128,-0.940645,0.441677,-0.357340,-0.107298,-0.786512,0.243289,1.054918,...,0.221208,0.696484,0.137578,0.623664,-0.296703,0.595952,-0.025197,-0.024916,19.99,0
80079,58323.0,1.079163,-0.263549,-0.184066,1.081574,0.181965,0.766329,-0.149275,0.334954,0.494358,...,-0.117856,-0.402465,-0.244360,-1.208110,0.677886,-0.255421,0.005757,-0.000571,71.95,0
69779,53577.0,1.102420,0.009591,1.147158,1.279351,-0.803017,-0.087742,-0.437259,0.102740,0.378050,...,0.168636,0.718317,-0.143203,0.589892,0.621207,-0.248999,0.061171,0.024774,15.13,0


In [55]:
sampled_data = pd.concat([legit_sample, fraud], axis=0)
sampled_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
217018,140728.0,-0.111629,0.619835,-0.004133,-0.063503,1.504045,-0.317948,0.840894,-0.189993,-0.080063,...,-0.345524,-0.892542,-0.054848,-0.097943,-0.596432,0.315761,-0.054834,-0.033327,1.98,0
210523,138020.0,0.094823,0.943427,-0.380045,-0.952696,1.655329,0.144880,1.332652,-0.463352,0.107920,...,0.240513,1.368728,-0.292101,-0.146433,-0.667049,-0.269276,0.288554,-0.073472,0.99,0
9387,13746.0,1.109824,0.331352,1.560526,2.719556,-0.483555,0.712093,-0.741679,0.236268,1.119022,...,-0.204147,-0.206828,0.000416,-0.067350,0.297569,-0.079282,0.013765,0.016206,0.00,0
46887,42975.0,0.443771,-2.268674,1.139574,0.125735,-2.150027,0.404790,-0.845121,0.114456,0.287907,...,0.403281,0.579466,-0.367342,0.502044,0.204558,-0.156440,0.017935,0.101540,426.72,0
96062,65591.0,1.209410,-0.441473,-0.984072,-1.041218,1.687524,3.300782,-0.702919,0.835463,0.174765,...,-0.263361,-1.067383,0.089452,1.032108,0.138759,0.752848,-0.076037,0.017357,64.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [62]:
sampled_data['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [63]:
sampled_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95002.075203,-0.021014,0.073480,0.029140,0.042734,0.120152,0.083157,0.061627,-0.015577,0.044465,...,-0.040156,0.013052,0.030023,0.012731,0.015302,-0.019656,0.014445,-0.006569,-0.015565,77.013577
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


## Model Building

* X = Feature Matrix (Independent Variables)
* y = Target Variable (Dependent Variables)

In [64]:
X = sampled_data.drop(columns = ['Class'], axis=1)
y = sampled_data['Class'] 

In [65]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
217018,140728.0,-0.111629,0.619835,-0.004133,-0.063503,1.504045,-0.317948,0.840894,-0.189993,-0.080063,...,0.122709,-0.345524,-0.892542,-0.054848,-0.097943,-0.596432,0.315761,-0.054834,-0.033327,1.98
210523,138020.0,0.094823,0.943427,-0.380045,-0.952696,1.655329,0.144880,1.332652,-0.463352,0.107920,...,0.372398,0.240513,1.368728,-0.292101,-0.146433,-0.667049,-0.269276,0.288554,-0.073472,0.99
9387,13746.0,1.109824,0.331352,1.560526,2.719556,-0.483555,0.712093,-0.741679,0.236268,1.119022,...,-0.168994,-0.204147,-0.206828,0.000416,-0.067350,0.297569,-0.079282,0.013765,0.016206,0.00
46887,42975.0,0.443771,-2.268674,1.139574,0.125735,-2.150027,0.404790,-0.845121,0.114456,0.287907,...,0.908301,0.403281,0.579466,-0.367342,0.502044,0.204558,-0.156440,0.017935,0.101540,426.72
96062,65591.0,1.209410,-0.441473,-0.984072,-1.041218,1.687524,3.300782,-0.702919,0.835463,0.174765,...,0.183201,-0.263361,-1.067383,0.089452,1.032108,0.138759,0.752848,-0.076037,0.017357,64.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [66]:
y

217018    0
210523    0
9387      0
46887     0
96062     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64

In [67]:
scaler = StandardScaler()
scaler.fit(X)

StandardScaler()

In [68]:
standardized_data =  scaler.transform(X)
standardized_data

array([[ 1.11522516,  0.4160295 , -0.33836569, ..., -0.13367822,
        -0.14829142, -0.44014624],
       [ 1.05808559,  0.45362067, -0.24926018, ...,  0.20178314,
        -0.24222234, -0.44460935],
       [-1.56413123,  0.63843363, -0.41780364, ..., -0.06666304,
        -0.03239421, -0.44907247],
       ...,
       [ 1.71917861,  0.31324199, -0.19888525, ...,  0.29610801,
         0.38445265, -0.09792906],
       [ 1.73215529, -0.13061624, -0.34772003, ...,  0.78433976,
        -0.66392033,  0.65543561],
       [ 1.7402156 ,  0.79905725, -0.46540739, ..., -0.07719124,
        -0.10613296, -0.25733888]])

In [69]:
X = standardized_data
X

array([[ 1.11522516,  0.4160295 , -0.33836569, ..., -0.13367822,
        -0.14829142, -0.44014624],
       [ 1.05808559,  0.45362067, -0.24926018, ...,  0.20178314,
        -0.24222234, -0.44460935],
       [-1.56413123,  0.63843363, -0.41780364, ..., -0.06666304,
        -0.03239421, -0.44907247],
       ...,
       [ 1.71917861,  0.31324199, -0.19888525, ...,  0.29610801,
         0.38445265, -0.09792906],
       [ 1.73215529, -0.13061624, -0.34772003, ...,  0.78433976,
        -0.66392033,  0.65543561],
       [ 1.7402156 ,  0.79905725, -0.46540739, ..., -0.07719124,
        -0.10613296, -0.25733888]])

## Train-Test Split

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=45)

In [71]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


In [72]:
print(y.shape, y_train.shape, y_test.shape)

(984,) (787,) (197,)


### Model Comparison

Evaluate the performance of different regression models on the given dataset. 
* Below we will evaluate the performance of 4 different regression models

In [73]:
models = [LogisticRegression, SVC, DecisionTreeClassifier, RandomForestClassifier]
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for model in models:
    classifier = model().fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))


In [74]:
metrics_df = pd.DataFrame({
    "Model": ["Logistic Regression", "SVM", "Decision Tree", "Random Forest"],
    "Accuracy": accuracy_scores,
    "Precision": precision_scores,
    "Recall": recall_scores,
    "F1 Score": f1_scores
})

metrics_df.set_index('Model', inplace=True)
metrics_df

,Accuracy,Precision,Recall,F1 Score
Model,,,,
Logistic Regression,0.949239,0.968421,0.929293,0.948454
SVM,0.939086,0.957895,0.919192,0.938144
Decision Tree,0.918782,0.919192,0.919192,0.919192
Random Forest,0.939086,0.948454,0.929293,0.938776


The Logistic Regression is the best model to use as it has the highest accuracy, precision, recall and F1 score.